# Fully offline Agent!

_(no Google Colab for this one since it is meant to be run entirely locally)_

This tutorial will guide on how to run agent, fully locally / offline i.e., running with a local LLM and a couple of local tools (Callable Python functions), so no data will leave your machine!

This can be especially useful for privacy-sensitive applications or when you want to avoid any cloud dependencies.

In this example, we will showcase how to let an agent read and write in your local filesystem! Specifically,
we will give read-access to the agent to our codebase so that it can write up and generate a README file to describe the project.

## Install Dependencies

any-agent uses the python asyncio module to support async functionality. When running in Jupyter notebooks,
 this means we need to enable the use of nested event loops. We'll install any-agent and enable this below using nest_asyncio.

In [1]:
%pip install 'any-agent[smolagents]' --quiet

import nest_asyncio

nest_asyncio.apply()

Note: you may need to restart the kernel to use updated packages.


## Set up your own LLM locally

Regardless of which agent framework you choose in any-agent, all of them support LiteLLM, which is a proxy that
allows us to use whichever LLM inside the framework, hosted on by any provider. For example,
we could use a local model via llama.cpp or [llamafile](https://github.com/Mozilla-Ocho/llamafile), a
google hosted gemini model, or a AWS bedrock hosted Llama model. For this example,
we will use [Ollama](https://ollama.com/) to run our LLM locally!



### Ollama setup

First, install Ollama by following their instructions: https://ollama.com/download

### Picking an LLM

Pick a model that you can run locally based on your hardware and download it from your terminal. For example:

16-24GB RAM -> `granite3.3:8b`  or  `gemma3:12b` with ~20–35k context length

24+GB RAM -> `mistral-small3.2:24b` or `devstral:24b` with ~40k+ context length

### Serving the model with the appropriate context length

By default, Ollama forces a context length of 8192 tokens to all models, which is not enough for our agent to work properly.
We can simply pass to the AgentConfig of `any-agent` as a model argument (`num_ctx` is ollama-specific) our desired value of context length like so:
```
    AgentConfig(
        model_id="ollama/granite3.3:8b",
        instructions="You must use the available tools to solve the task.",
        tools=[mcp_filesystem, show_plan],
        model_args={"num_ctx": 14000},
```

References: [AgentConfig](https://mozilla-ai.github.io/any-agent/api/config/), [num_ctx](https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values)

All four of the models above have a max context length of 128k tokens, but if you have limited RAM if you
set it to 128k it might cause you memory issues. For this example, we will set it to 14.000 tokens and provide a relatively small codebase.

### Load the model

Firstly, download the Ollama model locally:

```
ollama pull granite3.3:8b
```


## Configure the Agent and the Tools

First, let's setup which directory the agent should access.

In [1]:
from pathlib import Path

codebase_directory = "toy_codebase"
abs_path = str(Path(codebase_directory).resolve())
print(f"Codebase directory set: {abs_path}")

Codebase directory set: /home/kostis/MZAI/any-agent/docs/cookbook/toy_codebase


### Pick which tools to use

Since we want our agent to work fully locally/offline, we will not add any tools that require communication with remote
servers. In this example we are using python callable functions as tools, but we could also have used MCP servers that run fully locally (e.g. [mcp/filesystem](https://hub.docker.com/r/mcp/filesystem))

In [2]:
def read_files(directory: str) -> str:
    """
    Read all .py, .toml, .md files in the directory and all of its subdirectories recursively
    and return their content, separated by file path and name.

    Args:
        directory: The directory to search for files.
    """
    content_of_files = ""
    for ext in ["*.py", "*.toml", "*.md"]:
        for path in Path(directory).rglob(ext):
            if path.is_file():
                try:
                    with open(path, encoding="utf-8") as file:
                        content = file.read()
                    content_of_files += f"--- File: {path} ---\n{content}\n "
                except Exception as e:
                    content_of_files += f"--- File: {path} ---\nError reading file: {e}"
    return content_of_files


def write_file(directory: str, content: str) -> str:
    """
    Write content to a file.
    Args:
        directory: The directory where the file will be written.
        content: The content to write into the file.
    """
    file_path = directory + "/README.md"
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(content)
    return f"File written to {file_path}"

Now that you have downloaded your LLM and you have defined your tools, you need to pick your
agent framework to build your agent. Note that the agent you'll build with any-agent can be run across multiple
agent frameworks (Smolagent, TinyAgent, OpenAI, etc) and across various LLMs (Llama, DeepSeek, Mistral, etc).
For this example, we will use the smolagents framework.

In [3]:
from any_agent import AgentConfig, AnyAgent
from any_agent.tools import show_plan

# Define the agent
agent = AnyAgent.create(
    "smolagents",
    AgentConfig(
        model_id="ollama/granite3.3",
        instructions="""
        You must use the available tools to solve the task.
        """,
        tools=[write_file, read_files, show_plan],
        model_args={"num_ctx": 14000},
    ),
)

/home/kostis/MZAI/any-agent/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Run the Agent


In [6]:
agent_trace = agent.run(
    f"""
You are a technical documentation agent.
Carefully read all files in the project codebase.
Write to a file a clear, concise, and well-structured `README.md` that
explains the purpose, main features, and structure of the codebase.
The codebase directory is {abs_path}.
"""
)

╭────────────────────────────────────────── CALL_LLM: ollama/granite3.3 ──────────────────────────────────────────╮
│ ╭─ INPUT ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "role": "system",                                                                                       │ │
│ │     "content": "\n        You must use the available tools to solve the task.\n        "                    │ │
│ │   },                                                                                                        │ │
│ │   {                                                                                                         │ │
│ │     "role": "user",                                                                                         │ │
│ │     "content": "New task:\n\nYou are a technical documentation agent. \nCarefully read all files in the pro │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "read_files",                                                                              │ │
│ │     "tool.args": "{\"directory\": \"/home/kostis/MZAI/any-agent/docs/cookbook/toy_codebase\"}"              │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 525,                                                                                      │ │
│ │   "output_tokens": 38,                                                                                      │ │
│ │   "input_cost": 0,                                                                                          │ │
│ │   "output_cost": 0                                                                                          │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── EXECUTE_TOOL: read_files ────────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "directory": "/home/kostis/MZAI/any-agent/docs/cookbook/toy_codebase"                                     │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ --- File: /home/kostis/MZAI/any-agent/docs/cookbook/toy_codebase/src/workflow.py --- from pathlib import    │ │
│ │ Path                                                                                                        │ │
│ │                                                                                                             │ │
│ │ from loguru import logger from rapidfuzz import process                                                     │ │
│ │                                                                                                             │ │
│ │ from structured_qa.model_loaders import LlamaModel                                                          │ │
│ │                                                                                                             │ │
│ │ def get_matching_section(response, section_names): """ Use string similarity to find the most similar       │ │
│ │ section_name. """ return process.extractOne(response, section_names)[0]                                     │ │
│ │                                                                                                             │ │
│ │ def find_retrieve_answer( question: str, model: LlamaModel, sections_dir: str, find_prompt: str,            │ │
│ │ answer_prompt: str, max_sections_to_check: int | None = None, ) -> tuple[str, list[str]] | tuple[None,      │ │
│ │ list[str]]: """ Workflow to find the relevant section, retrieve the information, and answer the question.   │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │  Args:                                                                                                      │ │
│ │      question (str): The question to answer.                                                                │ │
│ │      model (LlamaModel): The model to use for generating completions.                                       │ │
│ │      sections_dir (str): The directory containing the sections.                                             │ │
│ │          See [`document_to_sections_dir`][structured_qa.preprocessing.document_to_sections_dir].            │ │
│ │          Structure of the sections directory:                                                               │ │
│ │                                                                                                             │ │
│ │          ```                                                                                                │ │
│ │          sections_dir/                                                                                      │ │
│ │              section_1.txt                                                                                  │ │
│ │              section_2.txt                                                                                  │ │
│ │              ...                                                                                            │ │
│ │          ```                                        

╭────────────────────────────────────────── CALL_LLM: ollama/granite3.3 ──────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "write_file",                                                                              │ │
│ │     "tool.args": "{\"directory\": \"/home/kostis/MZAI/any-agent/docs/cookbook/toy_codebase\", \"content\":  │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 4319,                                                                                     │ │
│ │   "output_tokens": 237,                                                                                     │ │
│ │   "input_cost": 0,                                                                                          │ │
│ │   "output_cost": 0                                                                                          │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── EXECUTE_TOOL: write_file ────────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "directory": "/home/kostis/MZAI/any-agent/docs/cookbook/toy_codebase",                                    │ │
│ │   "content": "# Toy Codebase\n\nA minimal example for the structured_qa project.\n\n## Structure\n\nThe cod │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ File written to /home/kostis/MZAI/any-agent/docs/cookbook/toy_codebase/README.md                            │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── CALL_LLM: ollama/granite3.3 ──────────────────────────────────────────╮
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ [                                                                                                           │ │
│ │   {                                                                                                         │ │
│ │     "tool.name": "final_answer",                                                                            │ │
│ │     "tool.args": "{\"answer\": \"A minimal example for the structured_qa project.\\n\\n## Structure\\n\\nTh │ │
│ │   }                                                                                                         │ │
│ │ ]                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ USAGE ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "input_tokens": 4909,                                                                                     │ │
│ │   "output_tokens": 203,                                                                                     │ │
│ │   "input_cost": 0,                                                                                          │ │
│ │   "output_cost": 0                                                                                          │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── EXECUTE_TOOL: final_answer ───────────────────────────────────────────╮
│ ╭─ Input ─────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ {                                                                                                           │ │
│ │   "answer": "A minimal example for the structured_qa project.\n\n## Structure\n\nThe codebase is organized  │ │
│ │ }                                                                                                           │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│ ╭─ OUTPUT ────────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ A minimal example for the structured_qa project.                                                            │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │                                                  Structure                                                  │ │
│ │                                                                                                             │ │
│ │ The codebase is organized as follows:                                                                       │ │
│ │                                                                                                             │ │
│ │  • /src contains the main Python source files.                                                              │ │
│ │  • /debug.md holds the original text from PDF when converted using pymupdf4llm.                             │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │                                                Preprocessing                                                │ │
│ │                                                                                                             │ │
│ │ The preprocessing.py script converts documents to a directory of sections.                                  │ │
│ │                                                                                                             │ │
│ │                                                                                                             │ │
│ │                                                Model Loaders                                                │ │
│ │                                                                                                             │ │
│ │ The model_loaders.py provides functions for loading various language models:                                │ │
│ │                                                                                                             │ │
│ │  • LlamaModel: Loads a Llama model from llama-cpp.                                                          │ │
│ │  • UnslothModel: Loads an Unsloth model using FastLanguageModel and a chat template.                        │ │
│ │  • GeminiModel: Loads a Gemini model using Google's generative AI.                                          │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## View the results 

The `agent.run` method returns an AgentTrace object, which has a few convenient attributes for displaying some interesting information about the run.

In [ ]:
print(agent_trace.final_output)  # Final answer
print(f"Duration: {agent_trace.duration.total_seconds():.2f} seconds")